In [1]:
import pandas as pd
from pandasql import PandaSQL
import warnings #handling warnings.
pd.options.mode.chained_assignment = None
warnings.filterwarnings('ignore')

In [2]:
productCatalogDf = pd.read_csv('../data/product_catalog_cleaned.csv')

In [3]:
productCatalogDf.head(2)

,Manufacturernumber,Articlenumber,EAN,Technical details,Picture normal reduced,Depth m,Width m,Length m,Weight kg,Delivery time days,...,Language,Manufacturername,Short_description_quality,Short_description_2_quality,Manufacturer_product_count,Product_length_category,Has_image,Volume_m3,Has_long_description,Description_length
0,0 601 6B4 000,06016B4000,NaN,§Titel§Akku-Tauchsäge BITURBO GKT 18V-52 GC Pr...,'https://www.nexmart.com/media/catalog/ampshar...,0.254,0.36,0.444,4.032,NaN,...,de,BOSCH,good,good,140,Medium,1,0.040599,1,1640
1,0 601 6B4 000,06016B4000,NaN,§Titel§Akku-Tauchsäge BITURBO GKT 18V-52 GC Pr...,'https://www.nexmart.com/media/catalog/ampshar...,0.254,0.36,0.444,4.032,NaN,...,en,BOSCH,good,bad,140,Medium,1,0.040599,0,0


In [4]:
productCatalogDf.tail(2)

,Manufacturernumber,Articlenumber,EAN,Technical details,Picture normal reduced,Depth m,Width m,Length m,Weight kg,Delivery time days,...,Language,Manufacturername,Short_description_quality,Short_description_2_quality,Manufacturer_product_count,Product_length_category,Has_image,Volume_m3,Has_long_description,Description_length
306,RALB2EU,RALB2EU,4.012079e+12,§Nennspannung§18§V|§Kapazität§5§Ah|§Ausführung...,'https://www.nexmart.com/media/catalog/ampshar...,NaN,NaN,NaN,0.64,NaN,...,de,BOSCH,good,bad,140,NaN,1,1.0,1,110
307,RALB2US,RALB2US,4.012079e+12,§Nennspannung§18§V|§Kapazität§5§Ah|§Ausführung...,'https://www.nexmart.com/media/catalog/ampshar...,NaN,NaN,NaN,0.64,NaN,...,de,BOSCH,good,bad,140,NaN,1,1.0,1,40


Which manufacturers have the biggest improvement potential in their data quality in absolute and relative numbers?

In [27]:
query = """
SELECT Manufacturername,
       COUNT(*) AS Total_Products,
       SUM(CASE WHEN Short_description_quality = 'bad' THEN 1 ELSE 0 END) + 
       SUM(CASE WHEN Short_description_2_quality = 'bad' THEN 1 ELSE 0 END) AS Total_Bad_Short_Descriptions
FROM productCatalogDf
GROUP BY Manufacturername
ORDER BY Total_Bad_Short_Descriptions DESC
"""
pandasql_instance = PandaSQL()
result = pandasql_instance(query, locals())
result

,Manufacturername,Total_Products,Total_Bad_Short_Descriptions
0,FEIN,100,100
1,GUSTAV KLAUKE GMBH,40,40
2,ROTHENBERGER,23,23
3,BOSCH,140,12
4,FISCHER,5,0


What product variable/column (description or property) usually contains data of good quality per manufacturer? And what is the % of good quality records per variable/column and manufacturer?

In [37]:
query_rel = """
SELECT Manufacturername,
       ((ROUND(100.0 * SUM(CASE WHEN Short_description_quality = 'good' THEN 1 ELSE 0 END) / COUNT(*), 2))+
       (ROUND(100.0 * SUM(CASE WHEN Short_description_2_quality = 'good' THEN 1 ELSE 0 END) / COUNT(*), 2)))/2 as Good_Quality_Product_Percentage
FROM productCatalogDf
GROUP BY Manufacturername
ORDER BY Good_Quality_Product_Percentage DESC;
"""

pandasql_instance = PandaSQL()
result_rel = pandasql_instance(query_rel, locals())
result_rel

,Manufacturername,Good_Quality_Product_Percentage
0,FISCHER,100.000
1,BOSCH,95.715
2,ROTHENBERGER,50.000
3,GUSTAV KLAUKE GMBH,50.000
4,FEIN,50.000
